<a href="https://colab.research.google.com/github/saspav/DA-104/blob/main/%D0%9F%D0%B0%D0%B2%D0%BB%D0%BE%D0%B2%D0%B0_%D0%A1%D0%92_%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81_%D1%81%D0%B0%D0%B9%D1%82%D0%BE%D0%B2_%D0%BD%D0%B0_Python_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D1%84%D1%80%D0%B5%D0%B9%D0%BC%D0%B2%D0%BE%D1%80%D0%BA%D0%B0_Scrapy_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача 1

Соберите информацию о заквасках с сайта pro-syr.ru (https://pro-syr.ru/zakvaski-dlya-syra/mezofilnye/)

Необходимо собрать следующие данные:
* Название продукта
* Цена
* Есть ли продукт в наличии

Результат должен быть записан в CSV файл

In [6]:
!pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 8.3 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 251 kB 34.3 MB/s 
     |████████████████████████████████| 47 kB 3.7 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 4.1 MB 50.0 MB/s 
     |████████████████████████████████| 3.1 MB 66.6 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=8903667f827f2c5c19ffa9aed36ac3e305243021339772d6a21e5681cf5f5e4f
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher


In [7]:
!scrapy startproject scrapy_project

Traceback (most recent call last):
  File "/usr/local/bin/scrapy", line 8, in <module>
    sys.exit(execute())
  File "/usr/local/lib/python3.7/dist-packages/scrapy/cmdline.py", line 154, in execute
    _run_print_help(parser, _run_command, cmd, args, opts)
  File "/usr/local/lib/python3.7/dist-packages/scrapy/cmdline.py", line 109, in _run_print_help
    func(*a, **kw)
  File "/usr/local/lib/python3.7/dist-packages/scrapy/cmdline.py", line 162, in _run_command
    cmd.run(args, opts)
  File "/usr/local/lib/python3.7/dist-packages/scrapy/commands/startproject.py", line 112, in run
    render_templatefile(tplfile, project_name=project_name, ProjectName=string_camelcase(project_name))
  File "/usr/local/lib/python3.7/dist-packages/scrapy/utils/template.py", line 9, in render_templatefile
    with open(path, 'rb') as fp:
FileNotFoundError: [Errno 2] No such file or directory: 'scrapy_project/scrapy_project/settings.py.tmpl'


In [8]:
# нужно сохранить код этой ячейки в файл 
# /content/scrapy_project/scrapy_project/spiders/zakvaski.py 

import scrapy


class ZakvaskiSpider(scrapy.Spider):
    name = 'Zakvaski'
    start_urls = ['https://pro-syr.ru/zakvaski-dlya-syra/mezofilnye/']

    def parse(self, response):
        for link in response.css('div.product-layout a::attr(href)'):
            yield response.follow(link.get(), callback=self.parse_page)

        last_page = response.css('ul.pagination a::attr(href)')[-1].get()
        last_page = int(last_page.split('=')[-1])
        for num_page in range(2, last_page + 1):
            link = f'https://pro-syr.ru/zakvaski-dlya-syra/mezofilnye/?page={num_page}'
            yield response.follow(link, callback=self.parse)

    @staticmethod
    def parse_page(response):
        price = response.css('.autocalc-product-price::text').get()
        price = price.split('руб')[0].replace(' ', '')
        yield {
            "name": response.css('#content h1::text').get(),
            "price": price,
            "in_stock": response.css('b.outstock::text').get()
        }
        

Автоматическое скачивание кода "паучка" из репозитария на гитхабе в нужную папку

In [9]:
!wget -P /content/scrapy_project/scrapy_project/spiders https://raw.githubusercontent.com/saspav/DA-104/main/scrapy_project/scrapy_project/spiders/zakvaski.py

--2022-08-28 09:08:05--  https://raw.githubusercontent.com/saspav/DA-104/main/scrapy_project/scrapy_project/spiders/zakvaski.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 996 [text/plain]
Saving to: ‘/content/scrapy_project/scrapy_project/spiders/zakvaski.py’

zakvaski.py         100%[===================>]     996  --.-KB/s    in 0s      

2022-08-28 09:08:05 (42.3 MB/s) - ‘/content/scrapy_project/scrapy_project/spiders/zakvaski.py’ saved [996/996]



In [10]:
!scrapy runspider /content/scrapy_project/scrapy_project/spiders/zakvaski.py -o "zakvaski.csv"

2022-08-28 09:08:06 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-08-28 09:08:06 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-08-28 09:08:06 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2022-08-28 09:08:06 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2022-08-28 09:08:06 [scrapy.extensions.telnet] INFO: Telnet Password: 48324a295e172ff1
2022-08-28 09:08:06 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-08-28 09:08:06